In [230]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import panel as pn
pn.extension('plotly')
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

### Loading the datasets

In [50]:
# Source: https://data.gov.ie/dataset/beq04-onstruction-sector-base-2015100-by-type-of-building-and-construction-year-and-statistic-fb86?package_type=dataset
# License: Creative Commons Attribution 4.0
#Accessed: 27 April 2023
total_production_indices_df = pd.read_csv("https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/BEQ04/CSV/1.0/en")

# Source: https://data.gov.ie/dataset/wpm28-wholesale-price-index-excl-vat-for-building-and-construction-materials?package_type=dataset
# License: Creative Commons Attribution 4.0
wholesale_price_index_df = pd.read_csv("https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/WPM28/CSV/1.0/en")

In [52]:
# Filter the DF to only contain price indices
wholesale_price_index_only_df = wholesale_price_index_df[wholesale_price_index_df['Statistic Label'] == 'Wholesale Price Index (Excl VAT) for Building and Construction Materials'].copy()

# Adding quarters from months
wholesale_price_index_only_df['Quarter'] = pd.to_datetime(wholesale_price_index_only_df['TLIST(M1)'].astype('str')+'01', format='%Y%m%d').dt.to_period('Q')

# Pivot the types of material to columns
construction_materials_price_df = wholesale_price_index_only_df.pivot(index=f'TLIST(M1)', columns='Type of Material', values='VALUE').reset_index()

# Adding quarters from months
construction_materials_price_df['Quarter'] = pd.to_datetime(construction_materials_price_df['TLIST(M1)'].astype('str')+'01', format='%Y%m%d').dt.to_period('Q').astype('str')
construction_materials_price_df.drop(columns=['TLIST(M1)'], axis=1, inplace=True)
construction_materials_price_df.head(2)

Type of Material,All other materials,All other metal fittings,All other products,Bituminous emulsions,Bituminous macadam and asphalt,"Bituminous macadam, asphalt and bituminous emulsions",Cement,Concrete blocks and bricks,Copper pipes and fittings,Electrical fittings,...,Rough timber (including plain sawn),Rough timber (softwood),Sand and gravel,Stone,"Stone, sand and gravel",Structural steel,Structural steel and reinforcing metal,Structural steel fabricated metal,Wooden windows and doors,Quarter
0,99.2,99.2,99.3,102.3,99.6,99.7,100.0,100.2,98.7,98.5,...,99.5,99.9,98.0,95.3,95.7,97.6,98.8,97.3,99.9,2015Q1
1,99.1,99.2,99.1,101.6,94.8,95.1,100.0,100.6,98.7,98.7,...,99.9,99.9,96.1,93.5,93.9,97.7,98.9,97.3,99.9,2015Q1


In [53]:
# Filter the production indices 
value_prod_index_df = total_production_indices_df[total_production_indices_df['Statistic Label'] == 'Value of Production Index in Building and Construction (Seasonally Adjusted)']
value_prod_index_df = value_prod_index_df.pivot(index='Quarter', columns='Type of Building and Construction', values='VALUE').reset_index()
value_prod_index_df.head(2)

Type of Building and Construction,Quarter,All building and construction,Building (excluding civil engineering),Civil engineering,Non-residential building,Residential building
0,2000Q1,148.962898,192.043996,58.781197,119.454102,297.635955
1,2000Q2,154.184897,197.648074,62.255066,121.366028,307.404645


In [54]:
# Join the materials cost DF with Value of production
value_prod_index_with_materials_cost_df = construction_materials_price_df.merge(value_prod_index_df) #.drop(columns=['Quarter'], axis=1)
value_prod_index_with_materials_cost_df.shape

(96, 46)

In [55]:
# Do same steps for the volume of production
volume_prod_index_df = total_production_indices_df[total_production_indices_df['Statistic Label'] == 'Volume of Production Index in Building and Construction (Seasonally Adjusted)']
volume_prod_index_df = volume_prod_index_df.pivot(index='Quarter', columns='Type of Building and Construction', values='VALUE').reset_index()

# Join the materials cost DF with volume of production
volume_prod_index_with_materials_cost_df = construction_materials_price_df.merge(volume_prod_index_df)  #.drop(columns=['Quarter'], axis=1)
volume_prod_index_with_materials_cost_df.shape

(96, 46)

## Regression Modeling

In [85]:
def perform_regression(model, X, y):
    # Divide the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit a multiple linear regression model using Scikit-learn
    regression_model = model.fit(X_train, y_train)
    
    # Evaluate the performance of the model using the testing set
    train_score = regression_model.score(X_train, y_train)
    test_score = regression_model.score(X_test, y_test)
    return [model, train_score, test_score]
#     print("Train R-squared score:", train_score)
#     print("Test R-squared score:", test_score)
    
#     y_pred = regression_model.predict(X_test)
#     residuals = y_test - y_pred

    # Display plots
#     fig, axes = plt.subplots(1,2, figsize=(30, 6), sharex=True)
#     axes[0].scatter(y_pred, residuals, label=str(model))
#     axes[0].axhline(y=0, color='r', linestyle='-')
#     axes[0].set_xlabel('Predicted Values')
#     axes[0].set_ylabel('Residuals')
#     axes[1].scatter(y_test, y_pred, label=str(model))
#     axes[1].plot(y_test, y_test, color='black', linestyle='--', label='Perfect prediction')
#     axes[1].set_xlabel('Actual Values')
#     axes[1].set_ylabel('Predicted Values')
#     axes[0].legend()
#     axes[1].legend()
#     plt.show()
    

In [92]:
# Linear Regression
X = value_prod_index_with_materials_cost_df.iloc[:, :-6]  # All Construction materials
y = value_prod_index_with_materials_cost_df['Residential building']   # Residential building
perform_regression(LinearRegression(), X, y)

[LinearRegression(), 0.8183220090293563, 0.29069747763593645]

In [94]:
print("Value of Production Index in Building and Construction (Seasonally Adjusted):")
X = value_prod_index_with_materials_cost_df.iloc[:, :-6]  # All Construction materials
# Run regression for each bulding types
regression_results = []
for dep_col in value_prod_index_with_materials_cost_df.columns[-5:]:
    y = value_prod_index_with_materials_cost_df[dep_col]
    regression_result = perform_regression(LinearRegression(), X, y)
    regression_result.insert(0, dep_col)
    regression_results.append(regression_result)
    
pd.DataFrame(regression_results, columns=['Target', 'Model', 'Training Score', 'Testing Score'])

Value of Production Index in Building and Construction (Seasonally Adjusted):


,Target,Model,Training Score,Testing Score
0,All building and construction,LinearRegression(),0.928764,0.090405
1,Building (excluding civil engineering),LinearRegression(),0.930123,0.347483
2,Civil engineering,LinearRegression(),0.823045,-0.272174
3,Non-residential building,LinearRegression(),0.942875,0.490490
4,Residential building,LinearRegression(),0.818322,0.290697


In [96]:
print("Volume of Production Index in Building and Construction (Seasonally Adjusted):")
X = volume_prod_index_with_materials_cost_df.iloc[:, :-6]  # All Construction materials
# Run regression for each bulding types
regression_results = []
for dep_col in volume_prod_index_with_materials_cost_df.columns[-5:]:
    y = volume_prod_index_with_materials_cost_df[dep_col]
    regression_result = perform_regression(LinearRegression(), X, y)
    regression_result.insert(0, dep_col)
    regression_results.append(regression_result)
    
pd.DataFrame(regression_results, columns=['Target', 'Model', 'Training Score', 'Testing Score'])

Volume of Production Index in Building and Construction (Seasonally Adjusted):


,Target,Model,Training Score,Testing Score
0,All building and construction,LinearRegression(),0.859231,-0.422585
1,Building (excluding civil engineering),LinearRegression(),0.885667,-0.174887
2,Civil engineering,LinearRegression(),0.687067,-0.874454
3,Non-residential building,LinearRegression(),0.895686,-0.014569
4,Residential building,LinearRegression(),0.805116,0.309172


Linear Regression is performing poorly for all building types. This is likely due to large number of construction materials, and not that many observations.

### Lasso Regression to help narrow down the most important construction materials

In [115]:
print("Value of Production Index in Building and Construction (Seasonally Adjusted):")
X = value_prod_index_with_materials_cost_df.iloc[:, :-6]  # All Construction materials

# Scaling or standardizing the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Run Lasso regression for each bulding types
regression_results = []
for dep_col in value_prod_index_with_materials_cost_df.columns[-5:]:
    y = value_prod_index_with_materials_cost_df[dep_col]
    regression_result = perform_regression(Lasso(), X_scaled, y)
    regression_result.insert(0, dep_col)
    regression_results.append(regression_result)
    
pd.DataFrame(regression_results, columns=['Target', 'Model', 'Training Score', 'Testing Score'])

Value of Production Index in Building and Construction (Seasonally Adjusted):


,Target,Model,Training Score,Testing Score
0,All building and construction,Lasso(),0.818960,0.706304
1,Building (excluding civil engineering),Lasso(),0.826494,0.716459
2,Civil engineering,Lasso(),0.566582,0.485965
3,Non-residential building,Lasso(),0.858174,0.756326
4,Residential building,Lasso(),0.584492,0.438152


With Lasso regression, the testing scores are improving to some extent. Let's perform Lasso with different alpha values.

In [107]:
regression_results = []
# Test Lasso regression with multiple alpha values
for alpha in np.logspace(-3, 3, 7):
    for dep_col in value_prod_index_with_materials_cost_df.columns[-5:]:
        y = value_prod_index_with_materials_cost_df[dep_col]
        regression_result = perform_regression(Lasso(alpha=alpha, max_iter=1000000), X_scaled, y)
        regression_result.insert(0, dep_col)
        regression_results.append(regression_result)
    
pd.DataFrame(regression_results, columns=['Target', 'Model', 'Training Score', 'Testing Score']).sort_values(['Training Score', 'Testing Score'], ascending=False).head(5)

,Target,Model,Training Score,Testing Score
3,Non-residential building,"Lasso(alpha=0.001, max_iter=1000000)",0.935543,0.521398
8,Non-residential building,"Lasso(alpha=0.01, max_iter=1000000)",0.932156,0.676293
0,All building and construction,"Lasso(alpha=0.001, max_iter=1000000)",0.919298,0.121003
1,Building (excluding civil engineering),"Lasso(alpha=0.001, max_iter=1000000)",0.918874,0.474752
5,All building and construction,"Lasso(alpha=0.01, max_iter=1000000)",0.915876,0.371168


In [116]:
# Divide the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
lasso = Lasso(alpha=0.001, max_iter=1000000).fit(X_train, y_train)

# Get the absolute values of coefficients from the model
coefficients = pd.Series(np.abs(lasso.coef_), index=X.columns)
# Print the top 15 coefficients with highest magnitude value
ordered_features = coefficients.sort_values(ascending=False)
print(ordered_features)

Structural steel                                        1248.992209
Structural steel fabricated metal                       1002.111083
Materials                                                323.507981
Other structural steel                                   228.083862
Reinforcing metal                                        150.733099
Structural steel and reinforcing metal                   102.306628
Rough timber (softwood)                                   97.726529
Plumbing materials including sanitary ware                43.902459
PVC pipes and fittings                                    39.650940
Ready mixed mortar and concrete                           38.636040
HVAC (heating and ventilation equipment)                  34.585077
Precast concrete                                          32.299066
All other products                                        32.178736
Other concrete products including precast                 26.864927
Other treated timber                            

Running LinearRegression with above fewer features only.

In [122]:
# Testing with different range of features in increment of 5
regression_results = []
for i in range(6,22,5):
    features = ordered_features[:i].keys()
    X = value_prod_index_with_materials_cost_df[features]
    # Run regression for each bulding types
    for dep_col in value_prod_index_with_materials_cost_df.columns[-5:]:
        y = value_prod_index_with_materials_cost_df[dep_col]
        regression_result = perform_regression(LinearRegression(), X, y)
        regression_result.insert(0, dep_col)
        regression_result.insert(0, i)
        regression_results.append(regression_result)

pd.DataFrame(regression_results, columns=['Features #', 'Target', 'Model', 'Training Score', 'Testing Score'])

,Features #,Target,Model,Training Score,Testing Score
0,6,All building and construction,LinearRegression(),0.437250,0.457061
1,6,Building (excluding civil engineering),LinearRegression(),0.422904,0.434204
2,6,Civil engineering,LinearRegression(),0.394910,0.412245
3,6,Non-residential building,LinearRegression(),0.472400,0.455922
4,6,Residential building,LinearRegression(),0.127939,0.103147
5,11,All building and construction,LinearRegression(),0.701853,0.337368
6,11,Building (excluding civil engineering),LinearRegression(),0.680509,0.217772
7,11,Civil engineering,LinearRegression(),0.569059,0.520141
8,11,Non-residential building,LinearRegression(),0.725748,0.313141
9,11,Residential building,LinearRegression(),0.426186,-0.088740


### Ridge Regression

In [124]:
# Testing with different range of features in increment of 5
regression_results = []
for i in range(6,22,5):
    features = ordered_features[:i].keys()
    X = value_prod_index_with_materials_cost_df[features]
    # Run regression for each bulding types
    for dep_col in value_prod_index_with_materials_cost_df.columns[-5:]:
        y = value_prod_index_with_materials_cost_df[dep_col]
        # Call the regression model
        regression_result = perform_regression(Ridge(), X, y)
        regression_result.insert(0, dep_col)
        regression_result.insert(0, i)
        regression_results.append(regression_result)

pd.DataFrame(regression_results, columns=['Features #', 'Target', 'Model', 'Training Score', 'Testing Score'])

,Features #,Target,Model,Training Score,Testing Score
0,6,All building and construction,Ridge(),0.425247,0.400305
1,6,Building (excluding civil engineering),Ridge(),0.413538,0.381877
2,6,Civil engineering,Ridge(),0.377515,0.369051
3,6,Non-residential building,Ridge(),0.466002,0.423278
4,6,Residential building,Ridge(),0.081470,-0.024173
5,11,All building and construction,Ridge(),0.687804,0.236309
6,11,Building (excluding civil engineering),Ridge(),0.669504,0.121918
7,11,Civil engineering,Ridge(),0.538771,0.456590
8,11,Non-residential building,Ridge(),0.717988,0.244506
9,11,Residential building,Ridge(),0.375583,-0.280853


### Model Visualization

In [259]:
def visualize_model(production_stat, model, scaler=None, gridsearchcv=False):
    # Filter the production related DF with selected statistic label
    production_indices_df = total_production_indices_df[total_production_indices_df['Statistic Label']==production_stat]
    
    # Pivot the type of building/construction values to columns
    pivoted_production_df = production_indices_df.pivot(index='Quarter', columns='Type of Building and Construction', values='VALUE').reset_index()
    
    # Join it to DF containing construction materials
    joined_df = construction_materials_price_df.merge(pivoted_production_df)
    
    # Define indepedent features
    X = joined_df.iloc[:, :-6]  # All Construction materials
    
    # Initiate lists to store all train and test scores for visualization
    train_scores =[]
    test_scores = []
    
    # Make 2 subplots
    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.25)
    scatter_scores = []
    
    # Loop through each building/construction type
    building_types = joined_df.columns[-5:]
    for dep_col in building_types:
        # Set the target type to building type
        y = joined_df[dep_col]
        # Divide the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Scale data if selected
        if scaler:
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.fit_transform(X_test)
            
        # GridSearchCV hyper-parameter tuning and cross-validation
        if gridsearchcv:
            print(model.__class__.__name__)
            # Define hyperparameter grid
            if model.__class__.__name__ in ['Ridge', 'Lasso']:
                param_grid = {'alpha': np.logspace(-3, 3, 7)}
                if model.__class__.__name__ == 'Lasso':
                    param_grid['max_iter'] = [10000]
            else:
                param_grid = {
                    'fit_intercept': [True, False]
                }
                
            # Define GridSearchCV object
            grid_search = GridSearchCV(model, param_grid, cv=5, return_train_score=True, scoring='r2') 

            # Fit GridSearchCV object to data
            grid_search.fit(X_train, y_train)
    
            # Use the best estimator Model
            regression_model = grid_search.best_estimator_

        else:
            # Fit a multiple linear regression model using Scikit-learn
            regression_model = model.fit(X_train, y_train)
        
        # Predict the target values using the trained model
        y_pred = regression_model.predict(X_test)

        # Evaluate the performance of the model using the testing set
        train_score = regression_model.score(X_train, y_train)
        test_score = regression_model.score(X_test, y_test)
        train_scores.append(train_score)
        test_scores.append(test_score)
        
        # Append the scatter plot to the list
        scatter_scores.append(go.Scatter(x=y_test, y=y_pred, mode='markers', name=dep_col, legendgroup='Predicted Values'))        
    
    # Add a line showing what perfect prediction should look like
    trace2_group2 = go.Scatter(x=y_test, y=y_test, mode='lines', name='Perfect Prediction', legendgroup='Predicted Values')
    
    # Plot the scores
    trace1_group1 = go.Bar(name=f'Training-{regression_model}', x=building_types, y=train_scores, legendgroup="Scores")
    trace1_group2 = go.Bar(name=f'Testing-{regression_model}', x=building_types, y=test_scores, legendgroup="Scores")
    
    # Add traces to the subplots
    fig.add_trace(trace1_group1, row=1, col=1)
    fig.add_trace(trace1_group2, row=1, col=1)
    
    # Add each Scatter plot to the sub-plot
    for scatter_score in scatter_scores:
        fig.add_trace(scatter_score, row=2, col=1)
    fig.add_trace(trace2_group2, row=2, col=1)
    
    # Update layout to give separate titles to each subplot
    fig.update_yaxes(title_text='R2 Score', row=1, col=1)
    fig.update_yaxes(title_text='Target Value', row=2, col=1)
    fig.update_xaxes(title_text='Predicted Value', row=2, col=1)
    
    # For grouped bars
    fig.update_layout(barmode='group',
                      title_text=f"{production_stat}-{regression_model}",
                     height=700, width=800,
                     legend=dict(
                        orientation='v',  # Set legend orientation to vertical
                        x=1.1,            # Adjust the x position of the legend
                        y=0.5            # Adjust the y position of the legend
                    ))
    return fig

In [262]:
# ridge_alphas = [Ridge(alpha=value) for value in np.logspace(-2, 2, 5)]
# lasso_alphas = [Lasso(alpha=value) for value in np.logspace(-2, 2, 5)]
# models = [LinearRegression(), *ridge_alphas, *lasso_alphas]
models = [LinearRegression(), Ridge(), Lasso()]
production_statistics = total_production_indices_df['Statistic Label'].unique()
scalers = [None, StandardScaler(), MinMaxScaler()]

# Define arguments for plotting function
kw = dict(production_stat=production_statistics, model=models, scaler=scalers, gridsearchcv=False)
i = pn.interact(visualize_model, **kw)
text = "<br>\n# Regresssion Models"

# Define plot Row and Column
p = pn.Row(pn.Column(text, i[0][0], i[0][1], i[0][2], i[0][3]), i[1][0])
p

LinearRegression 
LinearRegression 
LinearRegression 
LinearRegression 
LinearRegression

Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='production_stat', options=['Value of Production Inde...], value='Value of Production I...)
        [2] Select(name='model', options=[LinearRegression(), ...], value=LinearRegression())
        [3] Select(name='scaler', options=[None, StandardScaler(), ...])
        [4] Checkbox(name='gridsearchcv')
    [1] Plotly(Figure, name='interactive01186')